In [22]:
from sklearn.model_selection import ParameterGrid
%reload_ext autoreload
%autoreload 2
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
import convo as c 
import convL 
import modularized as m 
import batching as b 
import poolingL
import gin
import math 
import funcy as fy
import gridSearch as g 


In [7]:
import sys
sys.path.append("/content/drive/Othercomputers/Mein PC/Completed")

In [29]:
# create model with layers
dataset1 = b.GraphPropPredDataset(name='ogbg-molhiv')
dataset2 = b.GraphPropPredDataset(name='ogbg-molpcba')  

In [23]:
param_grid = {'num_layers': [1, 2, 3, 4, 5, 6],
              'learning_rate': [0.0001, 0.001, 0.01],
              'optimizer': ['Adam'],
              'regularization': ['DropOut', 'NodeSampling', 'DropEdge', 'GDC'],
              'probability': [0.3, 0.5, 0.7],
              'activation': ['relu', 'sigmoid', 'tanh'],
              'units': [32, 64],
              'convo_type': ['gcn', 'gin']}


sample = list(ParameterGrid(param_grid))[0:5]
sample 

type(list(ParameterGrid(param_grid)))

for entry in sample: 
    print(type(entry))


<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


In [30]:
list_models = list()
for entry in sample:
        activation = entry['activation']
        convo_type = entry['convo_type']
        learning_rate = entry['learning_rate']
        num_layers = entry['num_layers']
        optimizer = entry['optimizer']
        probability = entry['probability']
        regularization = entry['regularization']
        units = entry['units']
        list_models.append(g.create_model(activation, convo_type, learning_rate, num_layers, optimizer, probability, regularization, units,dataset1, False))
        
list_models


In [31]:
list_models[0].summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 X (InputLayer)                 [(None, 9)]          0           []                               
                                                                                                  
 num_nodes (InputLayer)         [(None,)]            0           []                               
                                                                                                  
 ref_A (InputLayer)             [(None,)]            0           []                               
                                                                                                  
 ref_B (InputLayer)             [(None,)]            0           []                               
                                                                                            

In [32]:
concrete_inputs = b.make_tf_datasets(dataset1)
training = concrete_inputs['train']

In [33]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=50)
history = list_models[0].fit(training.cache(), epochs  =100, callbacks = callback)


Epoch 1/100
    136/Unknown - 5s 16ms/step - loss: 1.5176WARNING:tensorflow:Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: loss
138/138 [==============================] - 5s 17ms/step - loss: 1.5107
Epoch 2/100
138/138 [==============================] - 2s 18ms/step - loss: 0.3528
Epoch 3/100
138/138 [==============================] - 2s 17ms/step - loss: 0.1879
Epoch 4/100
138/138 [==============================] - 2s 16ms/step - loss: 0.1544
Epoch 5/100
138/138 [==============================] - 2s 17ms/step - loss: 0.1442
Epoch 6/100
138/138 [==============================] - 2s 17ms/step - loss: 0.1416
Epoch 7/100
138/138 [==============================] - 2s 16ms/step - loss: 0.1412
Epoch 8/100
138/138 [==============================] - 2s 11ms/step - loss: 0.1398
Epoch 9/100
138/138 [==============================] - 2s 14ms/step - loss: 0.1393
Epoch 10/100
138/138 [==============================] - 2s 17ms/step - loss: 0.1400
Epoch

In [37]:
# Pooling 
wtf = poolingL.add_graph_idx(m.create_input(dataset1))
graph_idx = wtf['graph_idx']
graph_count = wtf['graph_count']